In [2]:
import xlearn as xl
import pandas as pd
import numpy as np
import gc
import time
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from pandas import DataFrame as DF
from keras.utils import to_categorical
from numpy import array
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [3]:
data = pd.read_table('/data/data/UserMovie_train.txt',header=None,encoding='gb2312',delim_whitespace=True,index_col=None)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data.head()

,0,1,2,3
0,uid,mid,timeStamp,star
1,1722994,1306505,2007-08-22,5.0
2,1405477,10574468,2013-04-24,4.0
3,15849871,4910186,2011-12-20,5.0
4,1068524,1304643,2006-02-07,3.0


In [17]:
data = pd.read_table('/home/ubuntu/data/UserMovie_train.txt',header=None,encoding='gb2312',delim_whitespace=True,index_col=None)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
"""
df = Dataframe to be converted to ffm format
Yype = train/test/val
Numerics=list of all numeric fields
Categories = list of all categorical fields
Features = list of all features except the Label and Id
"""
def convert_to_ffm(df,type,numerics,categories,features):
    currentcode = len(numerics)
    catdict={}
    catcodes={}
    #Flagging categorical and numerical fields
    for x in numerics:
        catdict[x]=0
    for x in categories:
        catdict[x]=1
    
    nrows = df.shape[0]
    ncolums=len(features)
    with open(str(type)+'_ffm.txt',"w") as text_file:
        #Looping over rows to convert each row to libffm format
        for n,r in enumerate(range(nrows)):
            datastring=""
            datarow=df.iloc[r].to_dict()
            datastring+=str(int(datarow['Label']))
            #For numerical fields,we are creating a dummy field here
            for i,x in enumerate(catdict.keys()):
                if(catdict[x]==0):
                    datastring=datastring+" "+str(i)+":"+str(i)+":"+str(datarow[x])
                else:
                    #For a new field appearing in a training example
                    if(x not in catcodes):
                        catcodes[x]={}
                        currentcode+=1
                        catcodes[x][datarow[x]]=currentcode #encdoing the feature
                    #For already encoded fields
                    elif(datarow[x] not in catcodes[x]):
                        currentcode+=1
                        catcodes[x][datarow[x]]=currentcode #encoding the feature
                    code = catcodes[x][datarow[x]]
                    datastring=datastring+" "+str(i)+":"+str(int(code))+":1"
                    
            datastring+='\n'
            text_file.write(datastring)


In [46]:
#数据预处理
result_list=list()
with open('/home/ubuntu/data/UserMovie_train.txt', 'r') as f:
    line = f.readline()
    result_list.append(line)
    line=f.readline()
    while line:
#         print(line)
        the_line=line.split('\t')
#         print(the_line)
        temp=''
        for i in range(len(the_line)):
            if i==2:
                temp+=the_line[i][0:4]+'\t'
            elif i==3:
                temp+=the_line[i][0]+'\n'
            else:
                temp+=the_line[i]+'\t'
#         print(temp)
        result_list.append(temp)
        line=f.readline()
#         break

In [47]:
with open('user_movie_processed.txt','w') as f:
    f.writelines(result_list)

In [12]:
1+2

3

In [ ]:
readin = open('/data/data/delete_final_data.txt', 'r')
#write data file
output = open('/data/ng/libsvm.txt', 'w')
try:
    the_line = readin.readline()
    while the_line:
        # delete the \n
        index=0
        the_line = the_line.strip('\n')
        output_line = ''
        temp=the_line.split('\t')
        print(temp)
        for i in range(len(temp)):
            #the label col
            if i == 0:
                output_line = temp[len(temp)-1]
            #the features cols
            if temp[i-1] != 'NULL' and i != 0:
                the_text = ' ' + str(i) + ':' + temp[i-1]
                output_line = output_line + the_text
        output_line = output_line + '\n'
        print(output_line)
        index+=1
        if(index>2):
            break
        output.write(output_line)
        the_line = readin.readline()
finally:
    readin.close()

In [52]:
da2 = pd.read_table('libsvm.txt',header=None,encoding='gb2312',delim_whitespace=True,index_col=None)
da2.head()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3
0,star,1:uid,2:mid,3:timeStamp
1,5,1:1722994,2:1306505,3:2007
2,4,1:1405477,2:10574468,3:2013
3,5,1:15849871,2:4910186,3:2011
4,3,1:1068524,2:1304643,3:2006


In [4]:
ffm_model = xl.create_ffm()
ffm_model.setTrain("criteo.tr.r100.gbdt0.ffm")
ffm_model.setValidate("criteo.va.r100.gbdt0.ffm")

param = {'task':'binary', # ‘binary’ for classification, ‘reg’ for Regression
         'k':2,           # Size of latent factor
         'lr':0.1,        # Learning rate for GD
         'lambda':0.0002, # L2 Regularization Parameter
         'Metric':'auc',  # Metric for monitoring validation set performance
         'epoch':25       # Maximum number of Epochs
        }

ffm_model.fit(param, "model.out")

Exception: ('Invalid key!', 'Metric')

In [ ]:
# Convert output to 0/1
ffm_model.setTest("criteo.va.r100.gbdt0.ffm")
ffm_model.setSigmoid()
ffm_model.predict("model.out", "output.txt")

In [3]:
data3 = pd.read_csv('/home/ubuntu/ng/FM/final_data.csv',sep=',')

In [4]:
data3=data3.sort_values(['uid','timeStamp'],ascending=True)

In [6]:
data3.head()


,uid,mid,star,timeStamp
61302,1000001,1301578,1,2005.0
99298,1000001,1293590,1,2005.0
190034,1000001,1309004,1,2005.0
195159,1000001,1292063,1,2005.0
335374,1000001,1292001,1,2005.0


In [5]:
# uid 48613  mid 72541
mid = data3['mid'].unique()
uid = data3['uid'].unique()
collect_mid_encoder = {}
collect_mid_dencoder = {}
collect_uid_encoder = {}
collect_uid_dencoder = {}
for i in range(len(mid)):
    collect_mid_encoder[i] = mid[i]
    collect_mid_dencoder[mid[i]] = i
for i in range(len(uid)):
    collect_uid_encoder[i] = uid[i]
    collect_uid_dencoder[uid[i]] = i
data3['mid'] = data3['mid'].map(lambda x:collect_mid_dencoder[x])
data3['uid'] = data3['uid'].map(lambda x:collect_uid_dencoder[x])

In [8]:
# user_matrix = np.zeros((len(uid),len(mid)),dtype='float16')
# item_matrix = np.zeros((len(mid),len(uid)),dtype='float16')
data3.head()

,uid,mid,star,timeStamp
61302,0,0,1,2005.0
99298,0,1,1,2005.0
190034,0,2,1,2005.0
195159,0,3,1,2005.0
335374,0,4,1,2005.0


In [91]:
def hash_star(x):
    if x>=3:
        return 1
    else:
        return 0

data3['star'] = data3['star'].apply(lambda x:hash_star(x))
# data3.groupby(['uid'])['star'].size()

In [9]:
# data3['timeStamp'] = data3['timeStamp'].apply(lambda x:str(x)[0:4])

In [6]:
data3=data3.dropna()

In [7]:
data3['timeStamp']=data3['timeStamp'].apply(lambda x:int(x)-2005)

In [12]:
# data3['timeStamp'].replace('nan','2006',inplace=True)

In [14]:
data3.iloc[11111111]

uid          31308
mid           4064
star             0
timeStamp        7
Name: 13989759, dtype: int64

In [15]:
data3.head()

,uid,mid,star,timeStamp
61302,0,0,1,0
99298,0,1,1,0
190034,0,2,1,0
195159,0,3,1,0
335374,0,4,1,0


In [22]:
uid_array=array(data3['uid'])
uid_encoded = to_categorical(uid_array)
# mid_array=array(data3['mid'])
# mid_encoded = to_categorical(mid_array)

MemoryError: 

In [148]:
np.concatenate((uid_encoded,mid_encoded,time_encoded),axis=1)

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [54]:
def my_one_hot(num,MAX,MIN=0):
    tag=[0]*(MAX-MIN+1)
    tag[num]=1
    return array(tag)

In [56]:
uid_array=array(data3['uid'])
mid_array=array(data3['mid'])

In [59]:
max_uid=max(uid_array)
min_uid=min(uid_array)
max_mid=max(mid_array)
min_mid=min(mid_array)

In [212]:
last_uid=-1
i=0
f=open('matric.txt','w')
# last_movie=[0]*(max_mid-min_mid+1)
for index, row in data3.iterrows():
#     print(row["uid"], row["mid"],row["star"],row["timeStamp"])
    uid_encoded=my_one_hot(row["uid"],max_uid,min_uid)
    mid_encoded=my_one_hot(row["mid"],max_mid,min_mid)
    if(last_uid==row["uid"]):
        last_movie=mid_encoded
    else:
        last_movie=[0]*(max_mid-min_mid+1)
#     print(uid_encoded.shape)
    time_encoded=array([row["timeStamp"]])
    t=np.concatenate((uid_encoded,mid_encoded,time_encoded,last_movie),axis=0)

    t=t.reshape(1,t.size)
#     print(t)
    strval = ','.join(str(x) for x in t[0])
    temp_write=strval+'\t'+str(row["star"])+'\n'
#     print(temp_write)
    f.write(temp_write)
    last_uid=row["uid"]
    i+=1
    if(i%1000==0):
        print(i/1000)
f.close()

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0


OSError: [Errno 28] No space left on device

In [97]:
t

array([1, 0, 0, ..., 0, 0, 0])

In [164]:
len(t)

3

In [193]:
da2 = pd.read_csv('matric.txt',header=None)
da2.head()

KeyboardInterrupt: 

In [187]:
t=np.arange(10).reshape(2,5)

In [189]:
t=t.reshape(1,t.size)
t

array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

In [173]:
strval = ','.join(str(x) for x in t[0])

In [208]:
ss=3
t=array([ss])
print(t)

[3]
